# LSTM for Sarcasm Detection

### Imports

In [16]:
import pandas as pd
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, CuDNNLSTM, Embedding, Dropout, Activation, Flatten, Bidirectional, GlobalMaxPool1D
from keras.models import Model, Sequential

Using TensorFlow backend.
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\elsas\Anaconda3\envs\Tech-Fundamentals-Lab1\lib\site-packages\IPython\core\interactiveshell.py", line 3326, in run_code
  File "<ipython-input-16-3579d8bf1ecb>", line 3, in <module>
    from keras.preprocessing.text import Tokenizer
  File "C:\Users\elsas\Anaconda3\envs\Tech-Fundamentals-Lab1\lib\site-packages\keras\__init__.py", line 3, in <module>
    from . import utils
  File "C:\Users\elsas\Anaconda3\envs\Tech-Fundamentals-Lab1\lib\site-packages\keras\utils\__init__.py", line 6, in <module>
    from . import conv_utils
  File "C:\Users\elsas\Anaconda3\envs\Tech-Fundamentals-Lab1\lib\site-packages\keras\utils\conv_utils.py", line 9, in <module>
    from .. import backend as K
  File "C:\Users\elsas\Anaconda3\envs\Tech-Fundamentals-Lab1\lib\site-packages\keras\backend\__init__.py", line 1, in <module>
    from .load_backend import epsilon
  File "C:\Users\elsas\Anaconda3\envs\Tech-Fundamentals-Lab1\lib\site-packages\keras\backend\l

ImportError: cannot import name 'context' from 'tensorflow.python.eager' (C:\Users\elsas\Anaconda3\envs\Tech-Fundamentals-Lab1\lib\site-packages\tensorflow_core\python\eager\__init__.py)

### Load data

In [10]:
df1 = pd.read_json("./data/Sarcasm_Headlines_Dataset.json", lines=True)
df2 = pd.read_json("./data/Sarcasm_Headlines_Dataset_v2.json", lines=True)
# re-order attibute columns in df2
df2 = df2[['article_link','headline','is_sarcastic']]
df = pd.concat([df1, df2], axis=0)
df = df.drop(['article_link'], axis=1)
print(len(df))
df.head()


55328


,headline,is_sarcastic
0,former versace store clerk sues over secret 'b...,0
1,the 'roseanne' revival catches up to our thorn...,0
2,mom starting to fear son's web series closest ...,1
3,"boehner just wants wife to listen, not come up...",1
4,j.k. rowling wishes snape happy birthday in th...,0


In [17]:
max_features = 10000
maxlen = 25
embedding_size = 200

tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(df['headline']))
X = tokenizer.texts_to_sequences(df['headline'])
X = pad_sequences(X, maxlen = maxlen)
y = df['is_sarcastic']

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\elsas\Anaconda3\envs\Tech-Fundamentals-Lab1\lib\site-packages\IPython\core\interactiveshell.py", line 3326, in run_code
  File "<ipython-input-17-4ad73b9b73a7>", line 5, in <module>
    tokenizer = Tokenizer(num_words=max_features)
NameError: name 'Tokenizer' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\elsas\Anaconda3\envs\Tech-Fundamentals-Lab1\lib\site-packages\IPython\core\interactiveshell.py", line 2040, in showtraceback
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\elsas\Anaconda3\envs\Tech-Fundamentals-Lab1\lib\site-packages\IPython\core\ultratb.py", line 1101, in get_records
  File "C:\Users\elsas\Anaconda3\envs\Tech-Fundamentals-Lab1\lib\site-packages\IPython\core\ultratb.py", line 319, in

NameError: name 'Tokenizer' is not defined

### Generate embeddings

https://nlp.stanford.edu/projects/glove/

### Split data into train, validation and test

We'll give 50% of the data to train and 25% each to validation and test.